In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle


import lightgbm as lgbm
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RandomizedSearchCV,KFold, GridSearchCV

from tensorflow.keras import layers, models
import tensorflow as tf

In [2]:
TRAIN = True  # Set to True for training; False for loading a pre-trained mode

In [3]:
from google.colab import files


uploaded = files.upload()

Saving option-chain-ED-NIFTY-19-Oct-2023 - Copy.csv to option-chain-ED-NIFTY-19-Oct-2023 - Copy (3).csv


In [4]:
df = pd.read_csv('option-chain-ED-NIFTY-19-Oct-2023 - Copy.csv')

In [5]:
df.head()

,Type,Volatility,STRIKE,CMP,risk_free,LTP,Expiry
0,CE,-,19550,19751.05,0.0685,215.85,0.013699
1,CE,3.58,19600,19751.05,0.0685,183.95,0.013699
2,CE,5.14,19650,19751.05,0.0685,143.55,0.013699
3,CE,6.24,19700,19751.05,0.0685,113.20,0.013699
4,CE,6.73,19750,19751.05,0.0685,86.00,0.013699


In [6]:
df = df.drop(0)

In [7]:
df.head()

,Type,Volatility,STRIKE,CMP,risk_free,LTP,Expiry
1,CE,3.58,19600,19751.05,0.0685,183.95,0.013699
2,CE,5.14,19650,19751.05,0.0685,143.55,0.013699
3,CE,6.24,19700,19751.05,0.0685,113.20,0.013699
4,CE,6.73,19750,19751.05,0.0685,86.00,0.013699
5,CE,7.02,19800,19751.05,0.0685,63.00,0.013699


In [8]:
df.head()

,Type,Volatility,STRIKE,CMP,risk_free,LTP,Expiry
1,CE,3.58,19600,19751.05,0.0685,183.95,0.013699
2,CE,5.14,19650,19751.05,0.0685,143.55,0.013699
3,CE,6.24,19700,19751.05,0.0685,113.20,0.013699
4,CE,6.73,19750,19751.05,0.0685,86.00,0.013699
5,CE,7.02,19800,19751.05,0.0685,63.00,0.013699


In [9]:
from sklearn.preprocessing import OneHotEncoder  # Add this import statement


In [10]:
target = "LTP"  # Replace with the actual target column name (Last Traded Price)
features = [
    "Volatility",
    "STRIKE",
    "CMP",
    "risk_free",
    "Expiry"
]

In [11]:
# Shuffle the dataset
df = df.sample(frac=1).reset_index(drop=True)

In [12]:
# Split the data into training and test sets
TRAIN_SIZE = 0.8
split_index = int(len(df) * TRAIN_SIZE)

In [13]:
train_df = df.iloc[:split_index, :].copy()
test_df = df.iloc[split_index:, :].copy()

# Prepare the training and test data
train_data = train_df[features].values
train_labels = train_df[target].values

test_data = test_df[features].values
test_labels = test_df[target].values

In [14]:
train_df = train_df.dropna(subset=[target])
train_data = train_df[features].values
train_labels = train_df[target].values

In [15]:
train_df.fillna(train_df.mean(), inplace=True)  # Replace NaNs with the mean

<ipython-input-15-79311e99fcc5>:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  train_df.fillna(train_df.mean(), inplace=True)  # Replace NaNs with the mean


In [20]:
# Create and train the LightGBM model
reg = lgbm.LGBMRegressor(
    n_estimators=1600,
    objective='mse',
    learning_rate=0.008,
    max_depth=8,
    reg_lambda=80,
    num_leaves=30
)

if TRAIN:
    reg.fit(train_data, train_labels)
    best_lgb_model = reg

    filename = 'best_lgb.sav'
    pickle.dump(best_lgb_model, open(filename, 'wb'))
else:
    filename = 'best_lgb.sav'
    best_lgb_model = pickle.load(open(filename, 'rb'))

print("LightGBM Model Parameters:")
print(best_lgb_model.get_params())

# Make predictions and calculate RMSE
lgbm_pred = best_lgb_model.predict(test_data)

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 34, number of used features: 0
[LightGBM] [Info] Start training from score 19.695588
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the spl

In [21]:
# Make predictions and calculate RMSE
lgbm_pred = best_lgb_model.predict(test_data)

print("LightGBM test RMSE:", mean_squared_error(lgbm_pred, test_labels) ** 0.5)

LightGBM test RMSE: 22.26155166735886


### Multi-layer Perceptron (Neural Nets)
